# SVD Cornac experiments
### 1. Init
#### 1.1. Import libraries

In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import sys
sys.path.append("../../")


import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

from IMPACT import utils
utils.set_seed(0)
from IMPACT import dataset
from IMPACT import model
from experiments.datasets.external_packages import cornac_util

import optuna
import gc
import json
import torch
import pandas as pd
from importlib import reload


from cornac.models import SVD
from cornac.metrics import MAE, RMSE, RatingMetric
import logging
import cornac

#### 1.2. Set up the loggers

In [2]:
utils.setuplogger(verbose = True, log_name="SVD_cornac")

#### 1.4. Parametrize the datasets

[INFO 19:13] #### postcovid ####
[INFO 19:13] #### config : {'seed': 0, 'load_params': False, 'save_params': False, 'embs_path': '../embs/postcovid', 'params_path': '../ckpt/postcovid', 'early_stopping': True, 'fast_training': True, 'learning_rate': 0.001, 'batch_size': 2048, 'num_epochs': 200, 'num_dim': 10, 'eval_freq': 1, 'patience': 30, 'device': 'cuda:0', 'lambda': 7.7e-06, 'tensorboard': False, 'flush_freq': True, 'prednet_len1': 128, 'prednet_len2': 64, 'best_params_path': '', 'num_layers': 0, 'version': 'pair', 'p_dropout': 0, 'low_mem_mode': True, 'user_nbrs_n': 10, 'item_nbrs_n': 5} ####


In [ ]:
# choose dataset here
dataset_name = 'postcovid'
version= ""#"_small"
# modify config here

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA is available. Using GPU.")
else:
    device = torch.device("cpu")
    print("CUDA is not available. Using CPU.")

config = {

    # General params
    'seed' : 0,

    # Saving params
    'load_params': False,
    'save_params': False,
    'embs_path' : '../embs/'+str(dataset_name),
    'params_path' :'../ckpt/'+str(dataset_name),

    # training mode
    'early_stopping' : True,
    'fast_training' : True, # (Only taken in account if early_stopping == true) If true, doesn't compute valid rmse PC-ER

    # Learning params
    'learning_rate': 0.001,
    'batch_size': 2048,
    'num_epochs': 200,
    'num_dim': 10, # for IRT or MIRT todo : is it necessary as we use concepts knowledge number as embedding dimension ?
    'eval_freq' : 1,
    'patience' : 30,
    'device': device,
    'lambda' : 7.7e-6,
    'tensorboard': False,
    'flush_freq' : True,

    # for NeuralCD
    'prednet_len1': 128,
    'prednet_len2': 64,
    'best_params_path':'',

    #For GCCD
    'num_layers': 0,
    'version': 'pair',
    'p_dropout': 0,
    'low_mem_mode' : True,
    'user_nbrs_n' : 10,
    'item_nbrs_n' : 5
}
concept_map = json.load(open(f'../datasets/{dataset_name}/concept_map.json', 'r'))
concept_map = {int(k):[int(x) for x in v] for k,v in concept_map.items()}
metadata = json.load(open(f'../datasets/{dataset_name}/metadata.json', 'r'))
utils.set_seed(config['seed'])
dataset_name += version
logging.info(f'#### {dataset_name} ####')
logging.info(f'#### config : {config} ####')

In [4]:
def generate_SVD(config,metadata) :
    return cornac.models.SVD(
        name="SVD",
        k=metadata['num_dimension_id'],
        max_iter=config['num_epochs'],
        learning_rate=config['learning_rate'],
        lambda_reg=config['lambda'],
        early_stop=True,
        num_threads=0,
        trainable=True,
        verbose=False,
        init_params=None,
        seed=config['seed'],
    )

### 2. CDM Training


In [ ]:
reload(utils)
reload(model)
reload(dataset)

seed = 0
utils.set_seed(0)

config['seed'] = seed
config['early_stopping'] = True
config['esc'] = 'objectives' #'loss' 'delta_objectives'
config['num_epochs']=200
config['eval_freq']=1
config['patience']=30

config['verbose_early_stopping'] = False
config["tensorboard"] = False
config['flush_freq'] = False
config['save_params']= False
config['disable_tqdm'] = True

In [ ]:
reload(cornac_util)

dataset_name = "postcovid"
eval_method,concept_map,metadata = cornac_util.load_dataset(dataset_name,config, 0)

study = optuna.create_study(
    directions=["minimize"],  # Specify directions for each objective
)
gc.collect()
torch.cuda.empty_cache()
study.optimize(lambda trial: cornac_util.objective_SVD(trial, config,metadata,eval_method,generate_SVD), n_trials=100, timeout=1800, n_jobs=4, gc_after_trial=True)

# Analyze the results
## requirements : plotly, nbformat
pareto_trials = study.best_trials
logging.info(f"Best trial for {dataset_name} : {study.best_trials}") 

logging.info("Number of trials :"+str(len(study.trials)))
for trial in study.trials:
    logging.info(f"Trial #{trial.number}")
    logging.info(f"  RMSE: {trial.values}")
    #logging.info(f"  DOA: {trial.values[1]}")
    logging.info(f"  Params: {trial.params}")

dataset_name = "promis"
eval_method,concept_map,metadata = cornac_util.load_dataset(dataset_name,config, 0)

study = optuna.create_study(
    directions=["minimize"],  # Specify directions for each objective
)
gc.collect()
torch.cuda.empty_cache()
study.optimize(lambda trial: cornac_util.objective_SVD(trial, config,metadata,eval_method,generate_SVD), n_trials=100, timeout=3600, n_jobs=4, gc_after_trial=True)

# Analyze the results
## requirements : plotly, nbformat
logging.info(f"Best trial for {dataset_name} : {study.best_trials}") 

logging.info("Number of trials :"+str(len(study.trials)))
for trial in study.trials:
    logging.info(f"Trial #{trial.number}")
    logging.info(f"  RMSE: {trial.values}")
    #logging.info(f"  DOA: {trial.values[1]}")
    logging.info(f"  Params: {trial.params}")

dataset_name = "movielens"
eval_method,concept_map,metadata = cornac_util.load_dataset(dataset_name,config, 0)

study = optuna.create_study(
    directions=["minimize"],  # Specify directions for each objective
)
gc.collect()
torch.cuda.empty_cache()
study.optimize(lambda trial: cornac_util.objective_SVD(trial, config,metadata,eval_method,generate_SVD), n_trials=100, timeout=3600, n_jobs=4, gc_after_trial=True)

# Analyze the results
## requirements : plotly, nbformat
logging.info(f"Best trial for {dataset_name} : {study.best_trials}") 

logging.info("Number of trials :"+str(len(study.trials)))
for trial in study.trials:
    logging.info(f"Trial #{trial.number}")
    logging.info(f"  RMSE: {trial.values}")
    #logging.info(f"  DOA: {trial.values[1]}")
    logging.info(f"  Params: {trial.params}")
    
dataset_name = "portrait"
eval_method,concept_map,metadata = cornac_util.load_dataset(dataset_name,config, 0)

study = optuna.create_study(
    directions=["minimize"],  # Specify directions for each objective
)
gc.collect()
torch.cuda.empty_cache()
study.optimize(lambda trial: cornac_util.objective_SVD(trial, config,metadata,eval_method,generate_SVD), n_trials=100, timeout=3600, n_jobs=4, gc_after_trial=True)

# Analyze the results
## requirements : plotly, nbformat
logging.info(f"Best trial for {dataset_name} : {study.best_trials}") 

logging.info("Number of trials :"+str(len(study.trials)))
for trial in study.trials:
    logging.info(f"Trial #{trial.number}")
    logging.info(f"  RMSE: {trial.values}")
    #logging.info(f"  DOA: {trial.values[1]}")
    logging.info(f"  Params: {trial.params}")


### 3. CDM Prediction
#### 3.1. Training and testing

In [5]:
def find_emb(algo):
    return algo.u_factors

In [7]:
reload(cornac_util)

dataset_name = "postcovid"
logging.info(dataset_name)
config['learning_rate'] = 0.00496
config['lambda'] = 0.0002607
metrics = cornac_util.test(dataset_name,config,generate_SVD,find_emb)
#
# dataset_name = "promis"
# logging.info(dataset_name)
# config['learning_rate'] = 0.00239
# config['lambda'] = 5.42e-6
# metrics = cornac_util.test(dataset_name,config,generate_SVD,find_emb)
#
# dataset_name = "movielens"
# logging.info(dataset_name)
# config['learning_rate'] = 0.00519
# config['lambda'] = 0.0001173
# metrics = cornac_util.test(dataset_name,config,generate_SVD,find_emb)

# dataset_name = "portrait"
# logging.info(dataset_name)
# config['learning_rate'] = 0.00224
# config['lambda'] = 0.0001569
# metrics = cornac_util.test(dataset_name,config,generate_SVD,find_emb)

[INFO 20:28] postcovid


/home/arthurb/.local/share/mamba/envs/liriscat/lib/python3.11/site-packages/cornac/data/dataset.py:335: UserWarning: 5748 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)
/home/arthurb/.local/share/mamba/envs/liriscat/lib/python3.11/site-packages/cornac/data/dataset.py:335: UserWarning: 712 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)
/home/arthurb/.local/share/mamba/envs/liriscat/lib/python3.11/site-packages/cornac/data/dataset.py:335: UserWarning: 719 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)
Rating: 100%|██████████| 10007/10007 [00:00<00:00, 192452.73it/s]
/home/arthurb/.local/share/mamba/envs/liriscat/lib/python3.11/site-packages/cornac/data/dataset.py:335: UserWarning: 5747 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)
/home/arthurb/.l

[INFO 20:35] rmse : 0.2295 +- 0.0015
[INFO 20:35] mae : 0.1744 +- 0.0014
[INFO 20:35] r2 : 0.5070 +- 0.0066
[INFO 20:35] pc-er : 0.0555 +- 0.0433
[INFO 20:35] doa : 0.5207 +- 0.0157
[INFO 20:35] rm : 0.0585 +- 0.0451
[INFO 20:35] rmse_round : 0.2447 +- 0.0017
[INFO 20:35] mae_round : 0.1725 +- 0.0018
